In [3]:
import argparse
import json
import random
import textwrap
from pathlib import Path

import dspy
from dspy.teleprompt import LabeledFewShot
from dspy.evaluate import Evaluate
from ccf.pylib.lm_data import PROMPT, Instance, TraitExtractor, score_prediction

from pprint import pp

In [9]:
args = argparse.Namespace(
    examples=Path("../data/fna/Asteraceae_examples_2025-03-17a.json"),
    model="ollama_chat/gemma3:27b",
    api_base="http://localhost:11434",
    api_key="",
    train_split=0.1,
    dev_split=0.5,
    test_split=0.4,
    seed=2203673,
)

In [10]:
random.seed(args.seed)

with args.examples.open() as f:
    example_data = json.load(f)

In [11]:
def example_splits(instances: list[Instance], train_split, dev_split):
    examples = [
        dspy.Example(text=i.text, prompt=PROMPT, traits=i.traits).with_inputs(
            "text", "prompt"
        )
        for i in instances
    ]
    random.shuffle(examples)

    total = len(examples)
    split1 = round(total * train_split)
    split2 = split1 + round(total * dev_split)

    dataset = {
        "train": examples[:split1],
        "dev": examples[split1:split2],
        "test": examples[split2:],
    }

    return dataset

In [12]:
instances = [Instance.dict_to_instance(d) for d in example_data]
dataset = example_splits(instances, args.train_split, args.dev_split)

lm = dspy.LM(args.model, api_base=args.api_base, api_key=args.api_key)
dspy.configure(lm=lm)

trait_extractor = dspy.Predict(TraitExtractor)

evaluator = Evaluate(
    devset=dataset["dev"],
    metric=score_prediction,
    num_threads=1,
    display_progress=True,
    display_table=True,
)

evaluator(trait_extractor, devset=dataset["dev"])

Average Metric: 180.56 / 203 (88.9%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [00:02<00:00, 68.07it/s]


2025/03/18 11:10:40 INFO dspy.evaluate.evaluate: Average Metric: 180.56017689206732 / 203 (88.9%)


,text,prompt,example_traits,pred_traits,score_prediction
0,"<b>Perennials,</b> 15–25 cm. <b>Stems</b> erect, puberulent to his...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='15–25 cm', leaf_shape='elliptic-lanceolate to...","Traits(plant_height='15–25 cm', leaf_shape='elliptic-lanceolate to...",✔️ [1.000]
1,"<b>Annuals,</b> 30–150(–400+) cm. <b>Stems</b> erect. <b>Leaves</b...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='30–150(–400+) cm', leaf_shape='rounded-deltat...","Traits(plant_height='30–150(–400+) cm', leaf_shape='rounded-deltat...",✔️ [0.929]
2,<b>Plants </b>10–80(–200) cm; nodal spines 0. <b>Leaves</b>: petio...,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–80(–200) cm', leaf_shape='suborbiculate to...","Traits(plant_height='10–80(–200) cm', leaf_shape='suborbiculate to...",✔️ [0.857]
3,"<b>Biennials </b>or perennials, 60–150(–200) cm; crown sprouts fro...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='60–150(–200) cm', leaf_shape='linear or linea...","Traits(plant_height='60–150(–200) cm', leaf_shape='linear or linea...",✔️ [0.752]
4,"<b>Annuals,</b> to 60 cm. <b>Stems</b> often spreading or prostrat...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='60 cm', leaf_shape='tapering to winged petiol...","Traits(plant_height='to 60 cm', leaf_shape='lanceolate to oblanceo...",✔️ [0.954]
...,...,...,...,...,...
198,"<b>Perennials,</b> 10–60 cm; taproots vertical, thick, fibrous, ca...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–60 cm', leaf_shape='oblanceolate, lyrate',...","Traits(plant_height='10–60 cm', leaf_shape='oblanceolate, lyrate t...",✔️ [0.927]
199,"<b>Perennials,</b> 10–60(–100+) cm. <b>Stems</b> erect. <b>Leaves<...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–60(–100+) cm', leaf_shape='deltate to lanc...","Traits(plant_height='10–60(–100+) cm', leaf_shape='deltate to lanc...",✔️ [0.929]
200,"<b>Perennials,</b> 50–300 cm (rhizomes often elongate, slender, pl...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='50–300 cm', leaf_shape='ovate to lanceolate',...","Traits(plant_height='50–300 cm', leaf_shape='broadly ovate to lanc...",✔️ [0.898]
201,<b>Plants </b>50–200 cm; rhizomes short- to long-creeping. <b>Stem...,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='50–200 cm', leaf_shape='tapering to bases; bl...","Traits(plant_height='50–200 cm', leaf_shape='oblanceolate (basal/m...",✔️ [0.709]


88.95

In [13]:
mipro_optimizer = dspy.MIPROv2(
    metric=score_prediction,
    auto="medium",
)

optimized_trait_extractor = mipro_optimizer.compile(
    trait_extractor,
    trainset=dataset["train"],
    max_bootstrapped_demos=4,
    requires_permission_to_run=False,
    minibatch=False
)

2025/03/18 11:14:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 32

2025/03/18 11:14:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/18 11:14:57 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/18 11:14:57 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 44%|████████████████████████████████████████████████████████████████▉                                                                                 | 4/9 [00:31<00:38,  7.78s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/19


 44%|████████████████████████████████████████████████████████████████▉                                                                                 | 4/9 [00:26<00:32,  6.56s/it]


...

2025/03/18 12:37:16 INFO dspy.evaluate.evaluate: Average Metric: 30.40097623223648 / 32 (95.0%)
2025/03/18 12:37:16 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 95.0 with parameters ['Predictor 0: Instruction 12', 'Predictor 0: Few-Shot Set 3'].
2025/03/18 12:37:16 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.82, 94.39, 92.77, 92.27, 90.14, 93.81, 93.96, 92.34, 89.94, 93.84, 95.0, 95.0, 91.8, 95.0, 93.8, 94.22, 93.93, 92.65, 94.92, 93.97, 89.54, 92.49, 95.0]
2025/03/18 12:37:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 95.0
2025/03/18 12:37:16 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/18 12:37:16 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 24 / 25 =====



Average Metric: 29.98 / 32 (93.7%): 100%|██████████████████████████████████████████████████████████████████████████████████████████| 32/32 [03:00<00:00,  5.63s/it]

2025/03/18 12:40:16 INFO dspy.evaluate.evaluate: Average Metric: 29.983525628326902 / 32 (93.7%)
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 93.7 with parameters ['Predictor 0: Instruction 12', 'Predictor 0: Few-Shot Set 14'].
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.82, 94.39, 92.77, 92.27, 90.14, 93.81, 93.96, 92.34, 89.94, 93.84, 95.0, 95.0, 91.8, 95.0, 93.8, 94.22, 93.93, 92.65, 94.92, 93.97, 89.54, 92.49, 95.0, 93.7]
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 95.0
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====



Average Metric: 30.40 / 32 (95.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 3306.67it/s]

2025/03/18 12:40:16 INFO dspy.evaluate.evaluate: Average Metric: 30.40097623223648 / 32 (95.0%)
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 95.0 with parameters ['Predictor 0: Instruction 12', 'Predictor 0: Few-Shot Set 3'].
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [87.82, 94.39, 92.77, 92.27, 90.14, 93.81, 93.96, 92.34, 89.94, 93.84, 95.0, 95.0, 91.8, 95.0, 93.8, 94.22, 93.93, 92.65, 94.92, 93.97, 89.54, 92.49, 95.0, 93.7, 95.0]
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 95.0
2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/03/18 12:40:16 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 95.0!


In [14]:
evaluator(optimized_trait_extractor, devset=dataset["dev"])

Average Metric: 193.20 / 203 (95.2%): 100%|██████████████████████████████████████████████████████████████████████████████████████| 203/203 [20:17<00:00,  6.00s/it]

2025/03/18 13:16:01 INFO dspy.evaluate.evaluate: Average Metric: 193.19519160250113 / 203 (95.2%)


,text,prompt,example_traits,pred_traits,score_prediction
0,"<b>Perennials,</b> 15–25 cm. <b>Stems</b> erect, puberulent to his...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='15–25 cm', leaf_shape='elliptic-lanceolate to...","Traits(plant_height='15–25 cm', leaf_shape='elliptic-lanceolate to...",✔️ [0.857]
1,"<b>Annuals,</b> 30–150(–400+) cm. <b>Stems</b> erect. <b>Leaves</b...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='30–150(–400+) cm', leaf_shape='rounded-deltat...","Traits(plant_height='30–150(–400+) cm', leaf_shape='rounded-deltat...",✔️ [0.857]
2,<b>Plants </b>10–80(–200) cm; nodal spines 0. <b>Leaves</b>: petio...,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–80(–200) cm', leaf_shape='suborbiculate to...","Traits(plant_height='10–80(–200) cm', leaf_shape='suborbiculate to...",✔️ [0.929]
3,"<b>Biennials </b>or perennials, 60–150(–200) cm; crown sprouts fro...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='60–150(–200) cm', leaf_shape='linear or linea...","Traits(plant_height='60–150(–200) cm', leaf_shape='linear or linea...",✔️ [0.837]
4,"<b>Annuals,</b> to 60 cm. <b>Stems</b> often spreading or prostrat...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='60 cm', leaf_shape='tapering to winged petiol...","Traits(plant_height='to 60 cm', leaf_shape='lanceolate to oblanceo...",✔️ [0.954]
...,...,...,...,...,...
198,"<b>Perennials,</b> 10–60 cm; taproots vertical, thick, fibrous, ca...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–60 cm', leaf_shape='oblanceolate, lyrate',...","Traits(plant_height='10–60 cm', leaf_shape='oblanceolate', leaf_le...",✔️ [0.968]
199,"<b>Perennials,</b> 10–60(–100+) cm. <b>Stems</b> erect. <b>Leaves<...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='10–60(–100+) cm', leaf_shape='deltate to lanc...","Traits(plant_height='10–60(–100+) cm', leaf_shape='deltate to lanc...",✔️ [0.857]
200,"<b>Perennials,</b> 50–300 cm (rhizomes often elongate, slender, pl...","What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='50–300 cm', leaf_shape='ovate to lanceolate',...","Traits(plant_height='50–300 cm', leaf_shape='broadly ovate to lanc...",✔️ [0.898]
201,<b>Plants </b>50–200 cm; rhizomes short- to long-creeping. <b>Stem...,"What is the plant size, leaf shape, leaf length, leaf width, leaf ...","Traits(plant_height='50–200 cm', leaf_shape='tapering to bases; bl...","Traits(plant_height='50–200 cm', leaf_shape='oblanceolate', leaf_l...",✔️ [0.891]


95.17

In [15]:
dspy.inspect_history(n=1)





[2025-03-18T13:16:01.896011]

System message:

Your input fields are:
1. `text` (str): the species description text
2. `prompt` (str): extract these traits

Your output fields are:
1. `traits` (Traits): the extracted traits

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## prompt ## ]]
{prompt}

[[ ## traits ## ]]
{traits}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "properties": {"deciduousness": {"type": "string", "default": "", "title": "Deciduousness"}, "elevation": {"type": "string", "default": "", "title": "Elevation"}, "fruit_length": {"type": "string", "default": "", "title": "Fruit Length"}, "fruit_type": {"type": "string", "default": "", "title": "Fruit Type"}, "fruit_width": {"type": "string", "default": "", "title": "Fruit Width"}, "habitat": {"type": "string", "default": "", "title": "Habitat"}, "leaf_length": {"type": "string", "default": "", "ti

In [17]:
optimized_trait_extractor.save("../data/fna/optimized_trait_extractor_2025-03-18a.pkl")

In [18]:
import pandas as pd

In [19]:
obj = pd.read_pickle("../data/fna/optimized_trait_extractor_2025-03-18a.pkl")

In [20]:
obj

{'lm': None,
 'traces': [],
 'train': [],
 'demos': [Example({'augmented': True, 'text': '<b>Annuals </b>or perennials, 10–70(–100) cm, herbaceous to suffrutescent. <b>Leaf</b> blades oblong to oblanceolate, mid-cauline (2.5–)4–13 mm wide. <b>Heads</b> essentially sessile, often surpassed by distal leaves that continue to very base of heads. <b>Involucres</b> (6–)10–15 × 12–30 mm. <b>Phyllaries</b> in 3–4 series, loose, subequal, apices erect to spreading, 1.3–1.7 mm wide, herbaceous. <b>Ray</b> florets 17–38; corollas 8.5–19 mm. <b>Disc</b> florets: corolla tubes longer than limbs. <b>2n</b> = 12.\nPhenology: Flowering (Jul–)Sep–Dec (sporadically Feb–Jun).\nHabitat: Sandy sites, along waterways, canal banks, dunes, beaches, salt flats\nElevation: 0–10 m', 'prompt': '\n    What is the plant size,\n    leaf shape, leaf length, leaf width, leaf thickness,\n    seed length, seed width,\n    fruit type, fruit length, fruit width,\n    deciduousness, phenology, habitat, elevation?\n    If i

In [21]:
optimized_trait_extractor.save("../data/fna/optimized_trait_extractor_2025-03-18a.json")

RuntimeError: Failed to save state to ../data/fna/optimized_trait_extractor_2025-03-18a.json with error: Traits(plant_height='10–70(–100) cm', leaf_shape='oblong to oblanceolate', leaf_length='', leaf_width='4–13 mm', leaf_thickness='', fruit_type='', fruit_length='', fruit_width='', seed_length='', seed_width='', deciduousness='', phenology='Flowering (Jul–)Sep–Dec (sporadically Feb–Jun)', habitat='Sandy sites, along waterways, canal banks, dunes, beaches, salt flats', elevation='0–10 m') is not JSON serializable. Your DSPy program may contain non json-serializable objects, please consider saving the state in .pkl by using `path` ending with `.pkl`, or saving the whole program by setting `save_program=True`.